# E-commerce Sales Data Cleaning Notebook

# Objective
This notebook processes and cleans the Online Retail dataset.
It removes incomplete or invalid rows, derives revenue-related fields,
and prepares the data for dashboarding and analytics.

# Import libraries

In [5]:
import pandas as pd
import os 
import re

# Load Dataset

In [6]:
raw_path = r'C:\Users\saipr\Downloads\ecommerce\data.csv'
df = pd.read_csv(raw_path,encoding='ISO-8859-1')

In [8]:
print(f"Original shape: {df.shape}")
df.head()

Original shape: (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Initial Exploration

In [10]:
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

# Data Cleaning

In [13]:
# Drop missing CustomerID
df.dropna(subset=['CustomerID'], inplace=True)

# Remove cancellations (InvoiceNo starting with 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Remove negative or zero Quantity and UnitPrice
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Feature Engineering

In [ ]:
# Add Revenue
df['Revenue'] = df['Quantity'] = df['UnitPrice']

# Add InvoiceWeek (Monday of that week)
df['InvoiceWeek'] = df['InvoiceDate'].dt.to_period('W').apply(lambda r: r.start_time)

# Reset index
df.reset_index(drop=True, inplace=True)

In [17]:
# Preview Cleaned Data
print(f"Cleaned shape: {df.shape}")
df.head()

Cleaned shape: (397884, 10)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,InvoiceWeek
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,2.55,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2.55,2010-11-29
1,536365,71053,WHITE METAL LANTERN,3.39,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,3.39,2010-11-29
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2.75,2010-11-29
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,3.39,2010-11-29
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,3.39,2010-11-29


# Save Cleaned Data

In [18]:
cleaned_path = r'C:\Users\saipr\Downloads\ecommerce\cleaned_retail.csv'
os.makedirs('data cleaned',exist_ok = True)
df.to_csv(cleaned_path, index=False)